In [1]:
/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:



URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT *
FROM Facilities
WHERE membercost =0


/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT( DISTINCT (facid) )
FROM Facilities
WHERE membercost =0



/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT *
FROM Facilities
WHERE membercost < monthlymaintenance * 0.2



/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid
BETWEEN 1
AND 5


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

select name, monthlymaintenance,
case when monthlymaintenance>100 then 'expensive'
else 'cheap' END as cost_binary
from Facilities


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT surname, firstname
FROM Members
WHERE joindate = (SELECT MAX( joindate ) FROM Members)


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

select distinct(m.surname), m.firstname,f.name  from Members m
join Bookings b on b.memid=m.memid
join Facilities f on f.facid= b.facid
where f.name like ('%Tennis%')


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */





/* Q9: This time, produce the same result as in Q8, but using a subquery. */


select t.* from
(select b.bookid, b.facid, b.memid, membercost, guestcost, concat(surname," ",firstname), starttime,
case when b.memid=0 then guestcost else membercost END as cost
from Bookings b

join Facilities f on f.facid=b.facid 
join Members m on m.memid=b.memid
where starttime > STR_TO_DATE('14,09,2012','%d,%m,%Y')
) t

where cost>30

         name
0    Bookings
1  Facilities
2     Members


In [1]:
# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

"""
database_management_system='sqlite'
username='student'
password='learn_sql@springboard
url='https://sql.springboard.com/'
database='country_club'
"""

# Create engine: engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

query1='''
select name
from sqlite_master
where type='table';
'''

tbl_df=pd.read_sql_query(query1,engine)
print(tbl_df)

         name
0    Bookings
1  Facilities
2     Members


In [7]:
#/* Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that theres a different cost for guests and members! */


query2='''
select sum(final_cost) as total_revenue, name from
(select b.bookid, b.facid, b.memid, f.guestcost, f.membercost, f.name,
case when memid=0 then f.guestcost
else f.membercost END as final_cost
from Bookings b
join Facilities f on f.facid=b.facid)t
group by name
having  sum(final_cost) >1000
order by total_revenue desc
'''

tbl_df2=pd.read_sql_query(query2,engine)
print(tbl_df2)

   total_revenue            name
0        20807.9  Massage Room 1
1         6987.3  Massage Room 2
2         4970.0    Squash Court
3         4205.0  Tennis Court 2
4         4040.0  Tennis Court 1


In [13]:
#/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

query2='''
select (m.surname|| ' ' ||m.firstname) as recommended_by, (r.surname|| ' ' ||r.firstname) as member  from Members m

join Members r on r.recommendedby=m.memid
where m.memid>0
order by member desc
'''

tbl_df2=pd.read_sql_query(query2,engine)
print(tbl_df2)


       recommended_by                   member
0         Smith Tracy  Worthington-Smyth Henry
1        Tracy Burton          Stibbons Ponder
2        Smith Darren               Smith Jack
3      Bader Florence         Sarwin Ramnaresh
4     Genting Matthew         Rumney Henrietta
5         Smith Tracy        Purview Millicent
6      Farrell Jemima             Pinker David
7        Smith Darren             Owen Charles
8        Smith Darren           Mackenzie Anna
9        Smith Darren          Joplette Janice
10        Jones David            Jones Douglas
11    Joplette Janice              Jones David
12  Purview Millicent                Hunt John
13     Butters Gerald          Genting Matthew
14    Joplette Janice               Dare Nancy
15        Smith Tracy            Crumpet Erica
16      Baker Timothy              Coplin Joan
17       Smith Darren           Butters Gerald
18         Rownam Tim               Boothe Tim
19     Farrell Jemima            Baker Timothy
20    Stibbon

In [17]:
#/* Q12: Find the facilities with their usage by member, but not guests */


query2='''
select count(distinct(bookid)) num_of_bookings,mem_name from

(select b.bookid,b.facid,b.memid, (m.surname|| ' ' ||m.firstname) as mem_name, b.starttime from Bookings b
 join Facilities f on f.facid=b.facid
 join Members m on b.memid=m.memid
 where b.memid>0) t

group by mem_name
'''

tbl_df2=pd.read_sql_query(query2,engine)
print(tbl_df2)



    num_of_bookings                 mem_name
0               120           Bader Florence
1               118               Baker Anne
2               166            Baker Timothy
3               188               Boothe Tim
4               163           Butters Gerald
5                53              Coplin Joan
6                 7            Crumpet Erica
7               117               Dare Nancy
8                34            Farrell David
9                80           Farrell Jemima
10               72          Genting Matthew
11               15                Hunt John
12              115              Jones David
13               14            Jones Douglas
14              159          Joplette Janice
15              126           Mackenzie Anna
16              131             Owen Charles
17               71             Pinker David
18               16        Purview Millicent
19              408               Rownam Tim
20               20         Rumney Henrietta
21        

In [18]:
#/* Q13: Find the facilities usage by month, but not guests */



query2='''
select count(distinct(bookid)) num_of_bookings, Month from

(select b.bookid,b.facid,b.memid, strftime('%m', b.starttime) as Month  from Bookings b
 join Facilities f on f.facid=b.facid
 join Members m on b.memid=m.memid
 where b.memid>0) t

group by Month
'''

tbl_df2=pd.read_sql_query(query2,engine)
print(tbl_df2)




   num_of_bookings Month
0              480    07
1             1168    08
2             1512    09
